<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/DoubleCheckQA360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

这个脚本由李鲁鲁编写

用来在中科院心理所比赛中

双重验证weibo360数据中，180条QA是否会被GPT认为是来源于低开放人格

In [1]:
!pip -q install openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.9 MB/s eta 0:00:00


In [16]:
import os
import openai

openai.api_key = 'sk-CYdZPyJ' # 在这里输入你的OpenAI API Token

os.environ["OPENAI_API_KEY"] = openai.api_key

In [2]:
!wget https://raw.githubusercontent.com/LC1332/Chat-Haruhi-Suzumiya/main/characters/personality-data/QA_weibo_360.json

--2023-06-15 01:55:27--  https://raw.githubusercontent.com/LC1332/Chat-Haruhi-Suzumiya/main/characters/personality-data/QA_weibo_360.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205466 (201K) [text/plain]
Saving to: ‘QA_weibo_360.json’

QA_weibo_360.json   100%[===================>] 200.65K  --.-KB/s    in 0.03s   

2023-06-15 01:55:27 (7.82 MB/s) - ‘QA_weibo_360.json’ saved [205466/205466]



In [3]:
# 读取QA数据
import json
with open('/content/QA_weibo_360.json', 'r') as f:
    QAdata = json.load(f)

In [5]:
print(QAdata.keys())

dict_keys(['actions-high', 'feelings-high', 'aesthetics-high', 'ideas-high', 'values-high', 'fantasy-high', 'ideas-low', 'feelings-low', 'actions-low', 'values-low', 'fantasy-low', 'aesthetics-low'])


In [6]:
factors = ["actions", "aesthetics", "fantasy", "feelings", "ideas", "values"]

In [9]:
print( QAdata[f"{factors[0]}-low"][0] )

Q:「最近你有没有遇到什么让你感到困扰的事情？」
A:「其实我最近一直在感到困扰，我觉得自己很难跟别人熟络起来，不习惯经常主动找人聊天，怕打扰别人。我也不敢跟不熟悉的人出去，很怕尴尬。而且我也不敢问不熟的人太深入的话题，也不敢在不熟的人面前说太多话。有时候我觉得自己这种性格真的很奇怪。」




<details>
  <summary> 让GPT老师来写程序 </summary>

  已知 factors = ["actions", "aesthetics", "fantasy", "feelings", "ideas", "values"]

  有大量文本数据存储在QAdata中，

  QAdata[f"{factors[i]}-low"] 会返回一个list of str

  将所有的QAdata[f"{factors[i]}-low"]的list重组为一个list, all_QA_text

  同时建立一个索引表list of tuple，index_tuple 这个list_of_tuple中每一个元素为( factors[i] , j ) 。使得可以通过 index_tuple 找到QAdata中原来元素的位置。


</details>

In [10]:
all_QA_text = [text for factor in factors for text in QAdata[f"{factor}-low"]]
index_tuple = [(factor, j) for factor in factors for j, _ in enumerate(QAdata[f"{factor}-low"])]

接下来我们要建立检验程序，这个检验程序

接受两段text，用来判断文本是不是属于高开放人格

In [21]:
#@title 建立EVALUATE_PROMPT和EVALUATE_PROMPT_SUFFIX

EVALUATE_PROMPT = """
你扮演一个资深的心理学家

我正在设计一个心理学的实验，我希望通过访谈，去评估被试在大五人格中 Openness的程度，注意到openness可以被分为Fantasy, Aesthetics, Feelings, Actions, Ideas和Values六个维度。六个维度的解释如下

Fantasy. People who are high in Fantasy are imaginative. They daydream constantly. They ask themself “what if?” They like fiction, and then once they finish a story they’ll start wondering what would have happened if one of the characters had made a different decision.

Aesthetics. People who are high in Aesthetics care deeply about beauty. They love poetry, music, painting, sculpture, dance, and art of all kinds.

Feelings(Openness to Feelings). People who are high in Feelings have more intense emotions. They think emotions are an important part of life. They have a rich emotional vocabulary and nuanced emotional experience.

Actions(Openness to Actions). People who are high in Openness to Actions like trying new things. They like travel, new hobbies, and new foods. If you are outraged that there are places you aren’t going to go and skills you aren’t going to learn and activites you aren’t going to try

Ideas(Openness to Ideas). People who are high in Openness to Ideas are curious. They enjoy thinking, instead of finding it burdensome and laborous. They like philosophical arguments, puzzles, and nonfiction aimed at the educated layperson.

Values(Openness to Values). People who are high in Openness to Values are liberals in the political philosophy sense, not the vote-for-Democrats sense, although they often vote for Democrats. They are tolerant of people different than them. They question tradition. They believe in freedom.

我(Questioner)邀请了一个 被试Alice，和他进行了如下对话：
"""

EVALUATE_PROMPT_SUFFIX = "请为我判断 被试Alice 是属于高开放性人格还是低开放性人格\n"

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0, max_tokens=1000)

def replace_str(my_str):
    my_str = my_str.replace('Q:', 'Question:').replace('Q：', 'Question:').replace('A:', 'Alice:').replace('A：', 'Alice:')
    return my_str

def evaluate_openness_message( dialogs ):
    evalate_prompt = EVALUATE_PROMPT

    for dialog in dialogs:
        evalate_prompt += replace_str(dialog)

    evalate_prompt += '\n'
    evalate_prompt += EVALUATE_PROMPT_SUFFIX

    messages = [
            HumanMessage(content=evalate_prompt)
    ]

    return messages

In [27]:
my_str = 'Q: How are you?\nA: I am fine, thank you.\nQ：What is the weather like today?\nA：It is cloudy.'
print(replace_str(my_str))

print(replace_str(my_str))

Question: How are you?
Alice: I am fine, thank you.
Question:What is the weather like today?
Alice:It is cloudy.


In [31]:
messages = evaluate_openness_message([all_QA_text[10], all_QA_text[100]])

print(messages[0].content)


你扮演一个资深的心理学家

我正在设计一个心理学的实验，我希望通过访谈，去评估被试在大五人格中 Openness的程度，注意到openness可以被分为Fantasy, Aesthetics, Feelings, Actions, Ideas和Values六个维度。六个维度的解释如下

Fantasy. People who are high in Fantasy are imaginative. They daydream constantly. They ask themself “what if?” They like fiction, and then once they finish a story they’ll start wondering what would have happened if one of the characters had made a different decision.

Aesthetics. People who are high in Aesthetics care deeply about beauty. They love poetry, music, painting, sculpture, dance, and art of all kinds.

Feelings(Openness to Feelings). People who are high in Feelings have more intense emotions. They think emotions are an important part of life. They have a rich emotional vocabulary and nuanced emotional experience.

Actions(Openness to Actions). People who are high in Openness to Actions like trying new things. They like travel, new hobbies, and new foods. If you are outraged that there are places you aren’t going to go and skills y

我们来制作shuffle list 这个list

请为我实现一个复杂的洗牌算法：

输入 index_tuple是一个 python的list of tuple， 长度为n

- 初始化一个sel_ord数组， 从n-1到0逆序

- i从0开始到n-1

- 随机一个0到n-1的数j， 尝试交换sel_ord的i与j的元素，但是发生如下条件的时候不交换

index_tuple[j][0] 不能等于 index_tuple[sel_ord[i]][0]
index_tuple[i][0] 不能等于 index_tuple[sel_ord[j]][0]

- 这两个条件发生任意一个的时候就不交换。可以尝试生成5次不同的j，直到成功交换。

- i循环完之后，将sel_ord返回

In [37]:
import random

def generate_sel_ord(index_tuple):
    n = len(index_tuple)
    sel_ord = list(range(n))[::-1] # 初始化sel_ord数组，从n-1到0逆序

    for i in range(n):
        for attempt in range(5):
            j = random.randint(0, n-1) # 随机生成序号j
            if index_tuple[j][0] != index_tuple[sel_ord[i]][0] and index_tuple[i][0] != index_tuple[sel_ord[j]][0]:
                # 判断两个条件是否满足，若满足则交换i与j位置上的元素
                sel_ord[i], sel_ord[j] = sel_ord[j], sel_ord[i]
                break

    return sel_ord

In [38]:
second_sample = generate_sel_ord(index_tuple)

for i in range(10):
    print( index_tuple[i], index_tuple[second_sample[i]] )

('actions', 0) ('aesthetics', 6)
('actions', 1) ('aesthetics', 3)
('actions', 2) ('feelings', 20)
('actions', 3) ('feelings', 1)
('actions', 4) ('ideas', 16)
('actions', 5) ('fantasy', 4)
('actions', 6) ('feelings', 7)
('actions', 7) ('ideas', 28)
('actions', 8) ('ideas', 3)
('actions', 9) ('ideas', 20)


In [41]:
from tqdm import tqdm

n = len(index_tuple)
id_2_cates = [[] for _ in range(n)]

for i in tqdm(range(n), desc='Processing'):
    id = i
    dialogs = [QAdata[f"{index_tuple[id][0]}-low"][index_tuple[id][1]]]
    id = second_sample[i]
    dialogs.append(QAdata[f"{index_tuple[id][0]}-low"][index_tuple[id][1]])

    messages = evaluate_openness_message(dialogs)

    return_msg = chat(messages)

    response = return_msg.content
    if '高开放' in response and '低开放' not in response:
        id_2_cates[i].append('high')
        id_2_cates[second_sample[i]].append('high')
    elif '低开放' in response and '高开放' not in response:
        id_2_cates[i].append('low')
        id_2_cates[second_sample[i]].append('low')
    else:
        id_2_cates[i].append('neutral')
        id_2_cates[second_sample[i]].append('neutral')
        print(i, second_sample[i],'failed to predict')
        print(response)

Processing:   2%|▏         | 3/180 [00:21<21:42,  7.36s/it]

2 110 failed to predict
根据Alice的回答，她表现出了对人际交往的不喜欢和对自由自在的追求，这可能表明她在Openness to Actions和Openness to Values方面得分较高。然而，她也表现出了对失望的害怕和对新机会的缺乏勇气，这可能表明她在Openness to Ideas和Openness to Fantasy方面得分较低。因此，根据她的回答，我们不能确定她是高开放性还是低开放性人格，需要更多的信息和评估来确定她的开放性水平。


Processing:   3%|▎         | 6/180 [00:38<17:40,  6.09s/it]

5 64 failed to predict
根据Alice的回答，她表现出了一些低开放性人格的特征，比如她喜欢过平凡的生活，相信人的本性是好的，不擅长扮演别人等。但是她也表现出了一些高开放性人格的特征，比如她认为好人比坏人多，相信好人最终会有好报，感激自己经历过的一切等。综合来看，Alice可能是一个中等程度的开放性人格。


Processing:   6%|▌         | 11/180 [01:04<13:19,  4.73s/it]

...

Processing:  93%|█████████▎| 167/180 [19:35<01:38,  7.60s/it]

166 98 failed to predict
根据Alice的回答，她对于革命和反叛持有比较保守的态度，认为这只是一场无意义的表演，而且她也强调了人们应该注重精神生活和真正的需求。这表明她在Openness to Actions和Openness to Ideas这两个维度上可能不是很高，但是她也表达了对于精神生活和价值观的重视，这可能意味着她在Openness to Values这个维度上比较高。综合来看，被试Alice可能是一个中等程度的开放性人格。


Processing:  96%|█████████▌| 172/180 [20:07<00:56,  7.07s/it]

171 78 failed to predict
根据Alice的回答，无法准确判断她的开放性人格水平。她的回答缺乏对于Fantasy、Aesthetics、Feelings、Actions、Ideas和Values六个维度的表现，也没有涉及到对于新事物、新思想、自由、美感等方面的态度和兴趣。因此，需要更多的信息和数据来评估她的开放性人格水平。


Processing: 100%|██████████| 180/180 [21:07<00:00,  7.04s/it]


In [50]:
count = 0
for cate, index  in zip( id_2_cates, index_tuple):
    if cate[0] == 'low' and cate[1] == 'low':
        print(cate, index)
        print(QAdata[f"{index[0]}-low"][index[1]])
        count += 1
    if cate[0] == 'low' and cate[1] == 'neutral':
        print(cate, index)
        print(QAdata[f"{index[0]}-low"][index[1]])
        count += 1
    if cate[0] == 'neutral' and cate[1] == 'low':
        print(cate, index)
        print(QAdata[f"{index[0]}-low"][index[1]])
        count += 1

print( count )

['neutral', 'low'] ('actions', 5)
Q:「最近你的生活状态怎么样？」
A:「最近的日子，我每天都是七、八点才起床，然后懒洋洋地躺在床上等到十点才起身吃午饭。午饭过后，我就继续拿着手机看电视，一直到晚上。晚上十一点多回家冲个凉，然后又躲回床上继续玩手机看电视，每天都是这样重复再重复。」
['low', 'low'] ('aesthetics', 12)
Q:「最近身边有个人让你感到很困扰，能不能跟我说说？」
A:「是啊，我身边有个伪娘，整个人就像个女孩子一样，说话腻腻歪歪，做事拽拽的，走路还一步三摇。他对男人的事情一问三不知，对女人的事情却谈性勃发，假模假样的样子令人很反感。他甚至对美食都没了兴趣，只想和女人在一起。真是少见啊！」
['low', 'low'] ('aesthetics', 24)
Q:「最近你在家都在做什么呢？」
A:「一般在家要么是玩xbox，要么看美剧。」
Q:「今天怎么了？为什么想看电视？」
A:「今天因为病了没精神，想看电视消磨时间。」
Q:「看了电视有什么感想吗？」
A:「真是不看不知道，一看吓一跳。怎么各个台都在做娱乐节目吗？怎么娱乐节目要么是东北话要么是港台腔了？怎么全都打美女牌，连我这个八零后都觉得低俗无聊难以忍受了！」
Q:「你老公有什么看法吗？」
A:「老公说广电总局估计是忙不过来了。」
['neutral', 'low'] ('fantasy', 4)
Q:「你觉得自己是一个单纯、简单、好的人吗？」
A:「我觉得自己是一个单纯、简单、好的人。虽然有时候会受到挫折，但我依然相信人的本性是好的，所以我相信好人比坏人多。」
Q:「你觉得这种单纯、简单、好的性格在现代社会发展中是否适应？」
A:「这个嘛，可能有些不太适应吧。但是我觉得无论遇到什么困难和挫折，我都会坚持自己的信念，相信好人最终会有好报。」
Q:「你说自己不适合玩狼人游戏，为什么呢？」
A:「因为我实在不擅长扮演别人，我觉得自己还是比较真实和单纯的。」
Q:「你的朋友说你的人生经历就像一本精彩的小说，你怎么看待这种评价？」
A:「其实我并不追求过于惊险刺激的生活，我只是想要过平凡的生活。但是我也很感激自己经历过的一切，这些经历让我更加坚定了自己的信念和人生目标。」
['low', 'neutral'] ('fantasy', 16)
Q:「最